In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import torch
import spektral
from spektral.datasets.citation import Citation
from torch.utils.data import Dataset, DataLoader

In [36]:
dataset = Citation("citeseer", random_split=True, normalize=True)
dataset[0]

c:\Users\Marco\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\sparse\_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Graph(n_nodes=3327, n_node_features=3703, n_edge_features=None, n_labels=6)

In [27]:
node_list = set([x[0] for x in edge_list] + [x[1] for x in edge_list])
len(node_list)

3327

In [40]:
node_ids = {k:v for v,k in enumerate(node_list)}

In [47]:
n = len(node_list)
adj_mat = np.zeros((n,n))

for x,y in edge_list:
    adj_mat[node_ids[x],node_ids[y]] = 1

In [60]:
dinv_mat = np.nan_to_num(np.diag(adj_mat.sum(axis=0)))

In [74]:
alpha = 0.15

Abar = np.matmul(adj_mat, dinv_mat)
I = np.diag(np.ones(n))
M = np.linalg.inv(I - (1-alpha)*Abar)

intimacy_mat = alpha*M

In [83]:
class CoraDataset(Dataset):
    def __init__(self, path='data/citeseer/', context_size=10) -> None:
        """
        Args:
            path (str, optional): path containing the Cora dataset. Defaults to 'data/citeseer/'.
            context_size (int, optional): number of nodes in a target node context, represent the
                                          topk nodes sorted by intimacy score. Defaults to 10.
        """
        super().__init__()

        self.context_size = context_size

        # swap nodes on each row as they are listed as target-source
        df = pd.read_csv(f"{path}/citeseer.cites", sep='\t', header=None)
        self.edge_list = [(x[1], x[0]) for x in df.values.tolist()]
        self.node_list = set([x[0] for x in self.edge_list] + [x[1] for x in self.edge_list])
        self.name_to_id = {k:v for v,k in enumerate(self.node_list)}
        self.id_to_name = {v:k for v,k in enumerate(self.node_list)}
        self.n = len(self.node_list)

        # load features and labels
        self.raw_features = pd.read_csv(f"{path}/citeseer.content", sep='\t', header=None, index_col=0)
        self.raw_features.index = self.raw_features.index.astype(str)
        self.labels = self.raw_features.iloc[:,-1].astype('category').cat.codes
        self.raw_features = self.raw_features.iloc[:,:-1]

        # pre-process graph to make data loader more efficient
        self.build_intimacy_matrix()
        self.build_contexts()

    def build_intimacy_matrix(self, alpha=0.15):
        # create adjacency matrix
        n = self.n
        adj_mat = np.zeros((n,n))
        name_to_id = self.name_to_id

        for x,y in self.edge_list:
            adj_mat[name_to_id[x], name_to_id[y]] = 1
            adj_mat[name_to_id[y], name_to_id[x]] = 1
        
        # compute inverse of diagonal degrees matrix
        dinv_mat = np.nan_to_num(np.diag(adj_mat.sum(axis=0)))

        # compute final matrix, for details see (1) in Graph-BERT by Zhang et al. '20 page 3
        Abar = np.matmul(adj_mat, dinv_mat)
        I = np.diag(np.ones(n))
        M = np.linalg.inv(I - (1-alpha)*Abar)

        self.intimacy_mat = alpha*M
    
    def build_contexts(self):
        context_list = np.zeros((self.n, self.context_size+1), dtype=np.int)

        # always include target node into its context
        context_list[:,0] = range(self.n)

        # context of a node contains the topk nodes sorted by intimacy score
        context_list[:,1:] = (-self.intimacy_mat).argsort(axis=1)[:,:self.context_size]
        
        self.context_list = context_list
    
    def __len__(self):
        """ each node together with its context represent an instance of the graph """
        return self.n
    
    def __getitem__(self, idx):
        context = self.context_list[idx,:]
        X = np.array([self.raw_features.loc[self.id_to_name[x]].to_numpy() for x in context])
        y = np.array([self.labels[self.id_to_name[x]] for x in context])
        return X, y

In [84]:
cora = CoraDataset()

c:\Users\Marco\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [85]:
cora[0]

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64),
 array([1, 4, 4, 1, 1, 1, 2, 2, 4, 5, 2], dtype=int8))

In [75]:
np.array([cora.raw_features.loc[cora.id_to_name[x]].to_numpy() for x in cora[0]]).sum()

311

In [63]:
len(cora.id_to_name), len(cora.raw_features)

(3327, 3312)

In [69]:
cora.raw_features.loc['63694'].to_numpy()

array([0, 0, 0, ..., 0, 0, 'Agents'], dtype=object)

In [16]:
features = np.genfromtxt('data/citeseer/citeseer.content', delimiter='\t')
features.shape

(3312, 3705)

In [30]:
features_df = pd.read_csv('data/citeseer/citeseer.content', sep='\t', header=None, index_col=0)
features_df.index = features_df.index.astype(str)
features_df.head()

c:\Users\Marco\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,1,2,3,4,5,6,7,8,9,10,...,3695,3696,3697,3698,3699,3700,3701,3702,3703,3704
0,,,,,,,,,,,,,,,,,,,,,
100157,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Agents
100598,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,IR
105684,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Agents
11099,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,DB
114091,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AI


In [40]:
labels = features_df.iloc[:,-1].astype('category').cat.codes

0
100157    1
100598    4
105684    1
11099     2
114091    0
dtype: int8

In [29]:
featuresfeatures_df.index.astype(str)

Index(['100157', '100598', '105684', '11099', '114091', '11510', '115971',
       '117999', '120432', '126894',
       ...
       'zhang01evolutionary', 'zhang01maximum', 'zhang01personalized',
       'zhang01pvm', 'zhang99evolving', 'zhang99query', 'zhang99situated',
       'zhang99towards', 'zhou00implementation', '455346'],
      dtype='object', name=0, length=3312)